In [2]:
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
import psycopg2
import pprint

In [3]:
conn_string = "host='localhost' dbname='nba_disappointments' user='postgres' password='password'"
conn = psycopg2.connect(conn_string)
c = conn.cursor()

In [3]:
for year in range(1996, 2022):
    with conn.cursor() as cur:
        cur.execute(f'''
        drop table player_scores_{year}
        ;
        ''')
    
    # Commit the changes to the database
    conn.commit()

In [4]:
for year in range(1996, 2022):
    with conn.cursor() as cur:
        cur.execute(f'''
        CREATE TABLE player_scores_{year} AS (
            select
                sub.uuid,
                sub.year,
                sub.team,
                sub.pos,
                sub.player,
                sub.salary,
                sub.percent_of_team,
                sub.wins as wins_contributed,
                case when sub.relative_wins = 0 then 0 else (sub.percent_of_team / sub.relative_wins)::decimal(6,2) end as production_score
                from (
                    with t1 as (
                        select 
                            avg(adjusted_wins)::Decimal(6,2) as avg_wins
                            from stats_{year}
                    ),

                    t2 as (
                        select
                            sal.uuid,
                            sal.year,
                            sal.team,
                            stat.pos,
                            sal.player,
                            salary,
                            (perc)::decimal(6,4) as percent_of_team,
                            perc_score as salary_score,
                            wins,
                            (adjusted_wins)::Decimal(6,2) as adjusted_wins
                            from team_salaries_{year} sal
                            left join stats_{year} stat
                                using(uuid)
                    )

                    select 
                        t2.*,
                        (t2.adjusted_wins / t1.avg_wins)::decimal(6,4) as relative_wins
                        from t1, t2
                ) sub
                where wins IS NOT NULL
                order by 8 desc
        );
        ''')
    
    # Commit the changes to the database
    conn.commit()

In [3]:
with conn.cursor() as cur:
    
    # Generate the SELECT statement to union all tables from 1996 to 2021
    select_query = ' UNION ALL '.join([f'SELECT * FROM player_scores_{year}' for year in range(1996, 2022)])
    
    # Insert the selected data from all tables into the new table
    cur.execute(f'''create table player_scores as (
    SELECT * FROM ({select_query}) AS all_scores);''')
    
    # Commit the changes to the database
    conn.commit()

# Close the database connection
conn.close()

In [9]:
print(' UNION ALL '.join([f'SELECT * FROM player_scores_{year}' for year in range(1996, 2022)]))

SELECT * FROM player_scores_1996 UNION ALL SELECT * FROM player_scores_1997 UNION ALL SELECT * FROM player_scores_1998 UNION ALL SELECT * FROM player_scores_1999 UNION ALL SELECT * FROM player_scores_2000 UNION ALL SELECT * FROM player_scores_2001 UNION ALL SELECT * FROM player_scores_2002 UNION ALL SELECT * FROM player_scores_2003 UNION ALL SELECT * FROM player_scores_2004 UNION ALL SELECT * FROM player_scores_2005 UNION ALL SELECT * FROM player_scores_2006 UNION ALL SELECT * FROM player_scores_2007 UNION ALL SELECT * FROM player_scores_2008 UNION ALL SELECT * FROM player_scores_2009 UNION ALL SELECT * FROM player_scores_2010 UNION ALL SELECT * FROM player_scores_2011 UNION ALL SELECT * FROM player_scores_2012 UNION ALL SELECT * FROM player_scores_2013 UNION ALL SELECT * FROM player_scores_2014 UNION ALL SELECT * FROM player_scores_2015 UNION ALL SELECT * FROM player_scores_2016 UNION ALL SELECT * FROM player_scores_2017 UNION ALL SELECT * FROM player_scores_2018 UNION ALL SELECT * FR

In [4]:
with conn.cursor() as cur:
    
    # Generate the SELECT statement to union all tables from 1996 to 2021
    select_query = ' UNION ALL '.join([f'SELECT * FROM season_totals_{year}' for year in range(1996, 2022)])
    
    # Insert the selected data from all tables into the new table
    cur.execute(f'''create table season_totals as (
    SELECT * FROM ({select_query}) AS all_scores);''')
    
    # Commit the changes to the database
    conn.commit()